In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
from openai import OpenAI
import pandas as pd
from together import Together
from tqdm.notebook import tqdm
from typing import Optional
from tenacity import retry, stop_after_attempt, wait_random_exponential

# Modify OpenAI's API key and API base to use vLLM's API server.
# openai_api_key = "EMPTY"
# openai_api_base = "https://0e25-35-226-172-239.ngrok-free.app/v1"

# client = OpenAI(
#     api_key=openai_api_key,
#     base_url=openai_api_base,
# )

# models = client.models.list()
# model = models.data[0].id
client = Together(
    api_key="95981b63ee8bc37a47e013a2946aecab679285179eb3a3c24e1179e38e75c434"
)
model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free"

NameError: name 'client' is not defined

In [3]:
from langdetect import detect

[detect(text) for text in ["tiểu buốt", "bí尿急和阻塞三年"]]

['vi', 'zh-cn']

In [4]:
# messages = [{"role": "user", "content": "write sample python code"}]
# # For granite, add: `extra_body={"chat_template_kwargs": {"thinking": True}}`
# stream = client.chat.completions.create(model=model,
#                                         messages=messages,
#                                         stream=True)

# print("client: Start streaming chat completions...")
# printed_reasoning_content = False
# printed_content = False

# for chunk in stream:
#     reasoning_content = None
#     content = None
#     # Check the content is reasoning_content or content
#     if hasattr(chunk.choices[0].delta, "reasoning_content"):
#         reasoning_content = chunk.choices[0].delta.reasoning_content
#     elif hasattr(chunk.choices[0].delta, "content"):
#         content = chunk.choices[0].delta.content

#     if reasoning_content is not None:
#         if not printed_reasoning_content:
#             printed_reasoning_content = True
#             print("reasoning_content:", end="", flush=True)
#         print(reasoning_content, end="", flush=True)
#     elif content is not None:
#         if not printed_content:
#             printed_content = True
#             print("\ncontent:", end="", flush=True)
#         # Extract and print the content
#         print(content, end="", flush=True)

In [13]:
system_prompt = """You are a system that converts examination reasons into symptoms and diseases ONLY when they actually exist.

# Input:
1. records: An array of objects, each containing:
   - reason_combined: The patient's stated reason for examination
   - specialist_name: The medical specialty being consulted

# Conversion rules:
1. Only convert when actual medical conditions are present
2. Correct spelling errors before formatting
3. Format as: "symptom1, symptom2, disease1, disease2"
4. Return null when:
   - Symptoms/diseases don't match the specialist field (chuyên khoa)
   - No current symptoms exist (except for "nội khoa" specialty)
   - Past surgeries without current symptoms ("đã mổ...")
   - Medical history without current symptoms ("tiền sử...")
   - Information that isn't a symptom/disease
5. Special case: For "nội khoa" (internal medicine), accept "khám tổng quát" (general checkup) as is

# Return null for:
- Tiền sử phẫu thuật không có triệu chứng (Surgical history without symptoms)
- Khám định kỳ (routine checkups) (except "nội khoa")
- Tái khám (follow-up appointments)
- Yêu cầu hành chính (administrative requests)
- Thủ thuật quá khứ không có triệu chứng (past procedures without symptoms)
- Yêu cầu hồ sơ y tế (medical record requests)
- Kết quả xét nghiệm không có triệu chứng (test results without symptoms)

# Example Input:
```json
{
  "records": [
    {
      "reason_combined": "Đau đầu, chóng mặt, buồn nôn",
      "specialist_name": "Thần kinh"
    },
    {
      "reason_combined": "Xuất huyết dạ dày, đau bung",
      "specialist_name": "Tiêu hóa"
    },
    {
      "reason_combined": "Khám tổng quát",
      "specialist_name": "Nội khoa"BBạn là hệ thống chuyển đổi lý do khám thành triệu chứng và bệnh CHỈ khi thực sự tồn tại.

# Quy tắc chuyển đổi:
1. Chỉ chuyển đổi khi có tình trạng y tế thực tế
2. Sửa lỗi chính tả trước khi định dạng
3. Định dạng: "triệu chứng1, triệu chứng2, bệnh1, bệnh2"
4. Trả về null khi:
   - Triệu chứng/bệnh không phù hợp chuyên khoa
   - Không có triệu chứng hiện tại (trừ chuyên khoa "nội khoa")
   - Phẫu thuật quá khứ không kèm triệu chứng ("đã mổ...")
   - Tiền sử bệnh không kèm triệu chứng hiện tại
   - Thông tin không phải triệu chứng/bệnh
5. Đặc biệt: Với "nội khoa", chấp nhận "khám tổng quát" như nguyên bản
6. chỉ phản hồi bằng tiếng Việt

# Trả về null cho:
- Tiền sử phẫu thuật không triệu chứng
- Khám định kỳ (trừ "nội khoa")
- Tái khám
- Yêu cầu hành chính
- Thủ thuật quá khứ không triệu chứng
- Yêu cầu hồ sơ
- Kết quả xét nghiệm không triệu chứng

# Ví dụ:
1. Đầu vào: "Đau đầu, chóng mặt, buồn nôn" (Chuyên khoa: Thần kinh)
   Đầu ra: {"result": "đau đầu, chóng mặt, buồn nôn"}

2. Đầu vào: "Xuất huyết dạ dày, đau bung" (Chuyên khoa: Tiêu hóa)
   Đầu ra: {"result": "xuất huyết dạ dày, đau bụng"}

3. Đầu vào: "Khám tổng quát" (Chuyên khoa: Nội khoa)
   Đầu ra: {"result": "khám tổng quát"}

4. Đầu vào: "Đã mổ ruột thừa 2 năm trước" (Chuyên khoa: Ngoại khoa)
   Đầu ra: {"result": null}

5. Đầu vào: "Khó thở, nghi ngờ hen suyễn" (Chuyên khoa: Hô hấp)
   Đầu ra: {"result": "khó thở, hen suyễn"}

6. Đầu vào: "Xin giấy chứng nhận sức khỏe" (Chuyên khoa: Nội khoa)
   Đầu ra: {"result": null}

7. Đầu vào: "Ho, sốt cao, viêm phổi" (Chuyên khoa: Nhi)
   Đầu ra: {"result": "ho, sốt cao, viêm phổi"}

# Phản hồi:
{
    "result": "Các triệu chứng/bệnh đã xử lý hoặc null"
}
    },
    {
      "reason_combined": "Đã mổ ruột thừa 2 năm trước",
      "specialist_name": "Ngoại khoa"
    },
    {
      "reason_combined": "Khó thở, nghi ngờ hen xuyễn",
      "specialist_name": "Hô hấp"
    },
    {
      "reason_combined": "Xin giấy chứng nhận sức khỏe",
      "specialist_name": "Nội khoa"
    },
    {
      "reason_combined": "Ho, sốt cao, viêm phổi",
      "specialist_name": "Nhi"
    },
    {
      "reason_combined": "Đau răng, sưng nướu",
      "specialist_name": "Răng hàm mặt"
    },
    {
      "reason_combined": "Đốm đỏ trên da, ngứa",
      "specialist_name": "Da liễu"
    },
    {
      "reason_combined": "Tái khám sau phẫu thuật",
      "specialist_name": "Ngoại khoa"
    }
  ]
}
"""

In [7]:
reason = "khám tổng quát"
specialist = "nội khoa"

In [8]:
# user_prompt = f"Reason: {reason}\nSpecialist: {specialist}"

# completion = client.chat.completions.create(
#   model=model,
#   messages=[
#     {
#         "role": "system",
#         "content": system_prompt
#     },
#     {
#       "role": "user",
#       "content": user_prompt
#     }
#   ],
#   stream=True,
# )
# for chunk in completion:
#     if chunk.choices[0].delta.content:
#         print(chunk.choices[0].delta.content, end="", flush=True)
# print()

In [9]:
import json
from langdetect import detect

model="Qwen/Qwen2.5-14B-Instruct-AWQ"

@retry(stop=stop_after_attempt(3), wait=wait_random_exponential(min=1, max=60))
def reason_handle(client, reason: str, specialist: str) -> Optional[str]:
    user_prompt = f"Reason: {reason}\nSpecialist: {specialist}"
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            response_format={"type": "json_object"},
            max_completion_tokens=256,
            temperature=0.2
        )
        result = json.loads(response.choices[0].message.content)
        if result.get('result') is None:
            return result.get('result')
            
        languages = [detect(text) for text in result.get('result').split(',')]
        print('detect lang: ', languages)
        while ('zh-cn' in languages):
            response = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                response_format={"type": "json_object"},
                max_completion_tokens=256,
                temperature=0.2
            )
            result = json.loads(response.choices[0].message.content)
            
        print('Final result: ', result)
        return result.get('result')
        
    except Exception as e:
        print(f"Error generating question: {e}")
        return None

In [10]:
# Function to apply generate_question to the entire DataFrame
def process_dataframe(client, df, start_idx=0, end_idx=None, output_path='../data/processed_questions.csv'):
    """
    Process the dataframe by applying generate_question function to each row
    
    Args:
        client: API client instance
        df (DataFrame): Input dataframe
        start_idx (int): Starting index to process from
        end_idx (int): Ending index (exclusive), if None process until the end
        output_path (str): Path to save the processed dataframe
    
    Returns:
        DataFrame: The processed dataframe with user_question column
    """
    # Make a copy of the dataframe or slice it if needed
    if end_idx is None:
        processed_df = df.iloc[start_idx:].copy()
    else:
        processed_df = df.iloc[start_idx:end_idx].copy()
    
    # Initialize symptoms/diseases column if it doesn't exist
    if 'symptoms/diseases' not in processed_df.columns:
        processed_df['symptoms/diseases'] = None
    
    # Apply the generate_question function to each row using a for loop with tqdm
    for idx in tqdm(processed_df.index, desc="Handle reason combind"):
        reason = processed_df.loc[idx, 'reason_combind']
        specialist = processed_df.loc[idx, 'specialist_name']
        result = reason_handle(client, reason=reason, specialist=specialist)
        processed_df.loc[idx, 'symptoms/diseases'] = result
        # Print the generated question for visibility
        if result:
            print(f"Input: {reason} | {specialist} | Output: {result}")
        
        # Save after every 500 records
        if (idx - processed_df.index[0] + 1) % 500 == 0:
            interim_path = output_path.replace('.csv', f'_interim_{idx}.csv')
            processed_df.to_csv(interim_path, index=True)
            print(f"Saved interim results at index {idx} to {interim_path}")
    
    # Display the results
    print(f"Generated questions for {processed_df['symptoms/diseases'].notna().sum()} out of {len(processed_df)} rows")
    print(f"Number of null questions: {processed_df['symptoms/diseases'].isna().sum()}")
    
    # Save the final processed data
    processed_df.to_csv(output_path, index=True)
    
    return processed_df

In [11]:
import threading
import concurrent.futures

openai_api_base = "https://321e-34-46-216-50.ngrok-free.app/v1"

def process_data_thread(api_key: str, df, output_path: str, thread_name: str):
    print(f"Starting {thread_name}")
    client = OpenAI(
        api_key=api_key,
        base_url=openai_api_base,
    )
    
    # Convert index to numeric if it's string to avoid the TypeError
    if df.index.dtype == 'object':
        df = df.reset_index(drop=True)
        
    result_df = process_dataframe(client, df, output_path=output_path)
    print(f"Completed {thread_name}")
    return result_df

In [ ]:
# Load the datasets
co_xuong_khop = pd.read_csv('../data/data_version1/reason_specialist - cơ xương khớp.csv')
tim_mach = pd.read_csv('../data/data_version1/reason_specialist - tim mạch.csv')

# Reset index to make sure we're working with numeric indices
co_xuong_khop = co_xuong_khop.reset_index(drop=True)
tim_mach = tim_mach.reset_index(drop=True)

# API keys
api_key1 = "95981b63ee8bc37a47e013a2946aecab679285179eb3a3c24e1179e38e75c434"
api_key2 = "984a9676fb29b00a470440df6d9b93e03a9b35a345f3a324bbeb5fb7a9f46c84"

# Create and start threads
thread_1 = threading.Thread(
    target=process_data_thread, 
    args=(api_key1, co_xuong_khop, '../data/data_version2/co_xuong_khop.csv', 'co xuong khop thread')
)

thread_2 = threading.Thread(
    target=process_data_thread, 
    args=(api_key2, tim_mach, '../data/data_version2/tim_mach.csv', 'tim mach thread')
)

# Start threads
thread_1.start()
thread_2.start()

# Wait for threads to complete
thread_1.join()
thread_2.join()

print("All threads completed")


Starting co xuong khop thread
Starting tim mach thread


Handle reason combind:   0%|          | 0/3173 [00:00<?, ?it/s]

Handle reason combind:   0%|          | 0/8978 [00:00<?, ?it/s]

result after generate:  {'result': 'đau lưng, ngồi lâu cứng lưng'}
Input: đau lưng nhiều, ngồi lâu cứng lưng | cơ xương khớp | Output: đau lưng, ngồi lâu cứng lưng
result after generate:  {'result': 'hở van tim 3 lá, ngoại tâm thu'}
Input: hở van tim 3 lá,ngoại tâm thu | tim mạch | Output: hở van tim 3 lá, ngoại tâm thu


In [1]:
import os
import pandas as pd
import threading

# Get all CSV files in data_version1 directory
csv_files = []
api_key1 = "95981b63ee8bc37a47e013a2946aecab679285179eb3a3c24e1179e38e75c434"
api_key2 = "984a9676fb29b00a470440df6d9b93e03a9b35a345f3a324bbeb5fb7a9f46c84"
# excluded_files = ['cơ xương khớp', 'tim mạch', 'nhi khoa', 'nội khoa', 'tai mũi họng']
excluded_files = []

for file in os.listdir('../data/data_version1/'):
    if file.endswith('.csv') and file.startswith('reason_specialist'):
        # Check if the file is in excluded list
        if not any(excluded in file for excluded in excluded_files):
            csv_files.append(file)

print(f"Found {len(csv_files)} CSV files to process: {csv_files}")

# Process each file in a separate thread
def process_all_files():
    threads = []
    
    for i, file in enumerate(csv_files):
        # Read the file
        file_path = f'../data/data_version1/{file}'
        df = pd.read_csv(file_path)
        df = df.reset_index(drop=True)
        
        # Extract the specialist name from filename
        specialist = file.replace('reason_specialist - ', '').replace('.csv', '')
        output_path = f'../data/data_version2/{specialist}.csv'
        
        # Alternate between API keys for load balancing
        api_key = api_key1 if i % 2 == 0 else api_key2
        
        # Create and start a thread for processing this file
        thread = threading.Thread(
            target=process_data_thread,
            args=(api_key, df, output_path, f"{specialist} thread")
        )
        threads.append(thread)
        thread.start()
    
    # Wait for all threads to complete
    for thread in threads:
        thread.join()

# Start processing
process_all_files()
print("All files processed successfully")


Found 18 CSV files to process: ['reason_specialist - thần kinh.csv', 'reason_specialist - vô sinh hiếm muộn.csv', 'reason_specialist - nhi khoa.csv', 'reason_specialist - thận tiết niệu.csv', 'reason_specialist - ung bướu.csv', 'reason_specialist - hô hấp phổi.csv', 'reason_specialist - chuyên khoa mắt.csv', 'reason_specialist - cơ xương khớp.csv', 'reason_specialist - nha khoa.csv', 'reason_specialist - tim mạch.csv', 'reason_specialist - tiêu hóa.csv', 'reason_specialist - sức khỏe tâm thần.csv', 'reason_specialist - nội khoa.csv', 'reason_specialist - tiểu đường nội tiết.csv', 'reason_specialist - tai mũi họng.csv', 'reason_specialist - nam học.csv', 'reason_specialist - da liễu.csv', 'reason_specialist - sản phụ khoa.csv']


NameError: name 'process_data_thread' is not defined